In [ ]:
# Importamos librerías a utilizar:
import pandas as pd
import json
import gzip
import ast
import csv
import gzip
import shutil
import os
import ast

**USER_REVIEWS**

In [ ]:
# Se procede a descomprimir el archivo:
archivo_comprimido = '/content/user_reviews.json.gz'
ruta_descomprimido = os.path.splitext(archivo_comprimido)[0]

with gzip.open(archivo_comprimido, 'rb') as f_in:
    with open(ruta_descomprimido, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print(f'Archivo descomprimido guardado en: {ruta_descomprimido}')

Archivo descomprimido guardado en: /content/user_reviews.json


In [ ]:
 with gzip.open('user_reviews.json.gz','r') as fin:
    for line in fin:
        print('user_reviews.json.gz', line)

In [51]:
# Creamos el DataFrame para user_reviews

def read_compressed_json(filname):
    data_list = []
    i = 0
    with gzip.open(filename, 'rb') as file:
        for line in file:
            decode_line = line.decode('utf-8')
            data_list.append(ast.literal_eval(decode_line.strip()))
            i = i + 1
            if (i==1000):
              break
        return data_list
filename = 'user_reviews.json.gz'
data = read_compressed_json(filename)
df_user_reviews = pd.DataFrame(data)

In [ ]:
# Visualizamos el dataframe:
display(df_user_reviews)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [ ]:
# Veamos si tiene valores nulos:
df_user_reviews.isnull().sum()

user_id     0
user_url    0
reviews     0
dtype: int64

In [52]:
# Como se puede ver en la pimera visualización, hay datos anidados en la columna "reviews" y necesitamos separarlos, por lo que se procede a utilizar el médoto explode o split, en esta oportunidad, utilizamos explode:

df_user_reviews_explode = df_user_reviews.explode('reviews')

df_user_reviews_explode.reset_index(drop=True, inplace=True)

# Utilizamos pd.json_normalize para crear columnas para cada clave en los diccionarios
df_user_reviews_explode = pd.concat([df_user_reviews_explode, pd.json_normalize(df_user_reviews_explode['reviews'])], axis=1)

# Eliminamos la columna 'reviews' original
df_user_reviews_explode.drop(columns=['reviews'], inplace=True)

df_user_reviews_explode.head(3)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


In [53]:
# Ahora nos quedaría trabajar con la columna "posted" ya que necesitamos "fecha" asi que la creamos:
df_user_reviews_explode['posted'] = df_user_reviews_explode['posted'].astype(str)
df_user_reviews_explode['fecha'] = df_user_reviews_explode['posted'].apply(lambda x: x[7:])

In [ ]:
# Veamos como quedaría:
df_user_reviews_explode.head(3)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,fecha
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,"November 5, 2011."
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,"July 15, 2011."
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,"April 21, 2011."


In [54]:
# Ahora procedemos a modificar el formato:
df_user_reviews_explode['fecha'] = df_user_reviews_explode['fecha'].str.rstrip('.')
df_user_reviews_explode['fecha'] = pd.to_datetime(df_user_reviews_explode['fecha'], errors='coerce', format='%B %d, %Y')
df_user_reviews_explode = df_user_reviews_explode.dropna(subset=['fecha'])
df_user_reviews_final = df_user_reviews_explode.drop(columns=['posted', 'funny'])
df_user_reviews_final.head(2)

,user_id,user_url,last_edited,item_id,helpful,recommend,review,fecha
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,22200,No ratings yet,True,It's unique and worth a playthrough.,2011-07-15


In [ ]:
df_user_reviews_final.head(2)


,user_id,user_url,last_edited,item_id,helpful,recommend,review,fecha
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,22200,No ratings yet,True,It's unique and worth a playthrough.,2011-07-15


In [55]:
# Visualizamos la info general:
df_user_reviews_explode.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928 entries, 0 to 2453
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      1928 non-null   object        
 1   user_url     1928 non-null   object        
 2   funny        1928 non-null   object        
 3   posted       1928 non-null   object        
 4   last_edited  1928 non-null   object        
 5   item_id      1928 non-null   object        
 6   helpful      1928 non-null   object        
 7   recommend    1928 non-null   object        
 8   review       1928 non-null   object        
 9   fecha        1928 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(9)
memory usage: 165.7+ KB


In [56]:
# Finalmente convertimos a csv
df_user_reviews_final.to_csv("df_user_reviews_final.csv")

In [57]:
# Importar la librería de archivos
from google.colab import files

# Descargar el archivo CSV
files.download('df_user_reviews_final.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**USER_ITEMS**

In [41]:
# Abrimos el archivo
def open_comp(archivo):
    data_list = []
    i = 0
    with gzip.open(archivo, 'rb') as file:
        for line in file:
            decode_line = line.decode('utf-8')
            data_list.append(ast.literal_eval(decode_line.strip()))
            i = i + 1
            if (i==1000):
              break
        return data_list

In [42]:
archivo = 'users_items.json.gz'
data = open_comp(archivo)
df_user_items = pd.DataFrame(data)

In [ ]:
# Visualizamos el archivo:
display(df_user_items)

In [43]:
# Observamos datos anidados en la columna "items" también, por lo que se procede a desanidar la columna con el método explode:#Descompone la lista en filas separadas
df_user_items_explode = df_user_items.explode('items')
df_user_items_explode.reset_index(drop=True, inplace=True)
# Creamos las nuevas columnas y concatenamos
df_user_items_explode = pd.concat([df_user_items_explode, pd.json_normalize(df_user_items_explode['items'])], axis=1)
# Se elimina el original y se reemplaza
df_user_items_explode.drop(columns=['items'], inplace=True)

df_user_items_explode.head(2)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0


In [44]:
# Veamos la información del archivo para visualzar el formato de los datos:
df_user_items_explode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89415 entries, 0 to 89414
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           89415 non-null  object 
 1   items_count       89415 non-null  int64  
 2   steam_id          89415 non-null  object 
 3   user_url          89415 non-null  object 
 4   item_id           89280 non-null  object 
 5   item_name         89280 non-null  object 
 6   playtime_forever  89280 non-null  float64
 7   playtime_2weeks   89280 non-null  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 5.5+ MB


In [ ]:
# Buscamos valores nulos:
df_user_items_explode.isnull().sum()

In [45]:
# Eliminamos valores nulos:
df_user_items_final = df_user_items_explode.dropna(subset=['item_id','playtime_forever'])

In [46]:
df_user_items_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89280 entries, 0 to 89414
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           89280 non-null  object 
 1   items_count       89280 non-null  int64  
 2   steam_id          89280 non-null  object 
 3   user_url          89280 non-null  object 
 4   item_id           89280 non-null  object 
 5   item_name         89280 non-null  object 
 6   playtime_forever  89280 non-null  float64
 7   playtime_2weeks   89280 non-null  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 6.1+ MB


In [47]:
# Convertimos el archivo a csv
df_user_items_final.to_csv('df_user_items_final.csv')

In [48]:
# Importar la librería de archivos
from google.colab import files

# Descargar el archivo CSV
files.download('df_user_items_final.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**STEAM_GAMES**

In [ ]:
def abrir_comprimido(archivo):
    data_list = []
    with gzip.open(archivo, 'rt', encoding='utf-8') as file:
        for line in file:
            try:
                data = json.loads(line.strip())
                data_list.append(data)
            except json.JSONDecodeError as e:
                print(f"Error en línea: {line.strip()}")
                print(e)
    return data_list

In [ ]:
# Creamos un DataFrame para steam_games y luego vemos la información
archivo = 'steam_games.json.gz'
data = abrir_comprimido(archivo)
df_steam_games = pd.DataFrame(data)

In [ ]:
# Con la función display mostramos el marco de datos del archivo
# A simple vista se logra visualizar datos nulos.
display(df_steam_games)

In [ ]:
# Utilizamos el método isnull() para ver si el df tiene valores nulos y los sumamos con sum(), el cuál se observan
# acontinuación en cada categoría:

df_steam_games.isnull().sum()

In [ ]:
# Comenzamos eliminando columnas:
drop_columns = ['user_id', 'steam_id', 'items', 'items_count']
df_steam_games.drop(drop_columns, axis=1, inplace=True)
df_steam_games.head(2)

In [ ]:
# Ahora eliminamos filas con valores nulos en sus columnas:
df_steam_games.dropna(how='all', inplace=True)

In [ ]:
# Luego verificamos filas con columnas de valores duplicados:
df_steam_games['id'].unique().size

In [ ]:
# Contamos el número de veces que aparece cada valor en la columna 'id'
conteo_ids = df_steam_games['id'].value_counts()

# Obtenemos los identificadores de los juegos que se repiten
ids_repitentes = conteo_ids[conteo_ids > 1].index.tolist()

# Imprimimos los identificadores de los juegos que se repiten y el número de veces que se repiten
print('Los siguientes identificadores de juegos se repiten:')
for id in ids_repitentes:
    print(f'- {id} ({conteo_ids[id]} repeticiones)')

In [ ]:
# Veamos la duplicación para el número de id repetido:
df_steam_games[df_steam_games['id']=='612880']

In [ ]:
# Eliminamos el índice:
indice_eliminado = df_steam_games[df_steam_games['id'] == '612880'].index[1]

# Eliminamos la fila
df_steam_games.drop(indice_eliminado, inplace=True)

In [ ]:
# Verificamos duplicados en otra columna:
df_steam_games['title'].duplicated().sum()

In [ ]:
# Agregamos la columna "precio" tal como se necesita:
df_steam_games.dropna(subset=['price'], inplace=True)
unique_values = df_steam_games['price'].unique()
unique_strings = [value for value in unique_values if isinstance(value, str)]

display(unique_strings)

In [ ]:
# Creamos una función para colocar el formato de la columna "precio" y colocar valor 0 donde haya un str
def precio_convert(valor):
    try:
        return float(valor)
    except ValueError:
        return 0.0

df_steam_games['price2'] = df_steam_games['price'].apply(precio_convert)

In [ ]:
# Agregamos el año a la columna "release_date"
df_steam_games['releasedate2'] = df_steam_games['release_date'].apply(pd.to_datetime, errors='coerce')
df_steam_games.dropna(subset=['releasedate2'], inplace=True)
df_steam_games['Año'] = df_steam_games['releasedate2'].dt.year

In [ ]:
# Se siguen observando datos nulos, procedemos a eliminar de las columnas:
df_steam_games.dropna(subset=['developer'], inplace=True)

In [ ]:
# Eliminar nulos y cambiar de tipo de formato a la columma
df_steam_games.dropna(subset=['genres'], inplace=True)
df_steam_games['genres'] = df_steam_games['genres'].astype('str')

In [ ]:
# Luego optimizamos más y eliminamos columnas que no nos sirven:
df_steam_games.drop(columns=['tags', 'reviews_url', 'discount_price', 'early_access', 'metascore', 'price', 'release_date'], inplace=True)
df_steam_games.head(3)

In [ ]:
# Finalmente convertimos el archivo a csv
df_steam_games.to_csv('df_steam_games_final.csv')

***1° FUNCIÓN: DEF USER_DATA***

In [ ]:
def userdata(user_id):
    df_user_items_id = df_user_items_final[df_user_items_final['user_id']==user_id]['item_id']
    respuesta = ''

    items = df_user_items_id.values
    precios =  []
    for i in items:
        precio = df_steam_games[df_steam_games['id']== i]['price2']
        if not precio.empty:
            precios.append(precio.values[0])

    df_user_reviews_id = df_user_reviews_final[df_user_reviews_final['user_id']==user_id]['recommend']

    j = 0
    for i in df_user_reviews_id.values:
        if(i == True):
            j = j + 1

    recomendacion = j/len(df_user_reviews_id.values)*100

    suma = 0

    for precio in precios:
        suma += precio

    item_count = df_user_items_final[df_user_items_final['user_id']=='js41637']['items_count'].iloc[0]

    respuesta = {'recomendación:': recomendacion,
                'Cantidad de dinero:' : suma,
                'Cantidad de items': item_count}

    return respuesta

In [ ]:
userdata('js41637')

***2° FUNCIÓN: DEF_COUNTREVIEWS***

In [ ]:
def count_reviews(fecha1, fecha2):
    df_user_fechas=df_user_reviews_final[['user_id']][df_user_reviews_final['fecha']
                                                  .between(fecha1, fecha2)].value_counts()

    primeros_valores = df_user_fechas.index.get_level_values('user_id').tolist()

    recomendaciones = []
    for i in primeros_valores:
        recomendacion = (userdata(i)[list(userdata(i).keys())[0]])
        recomendaciones.append(recomendacion)

    respuesta = {'usuario': primeros_valores, 'recomendación':recomendaciones}

    df = pd.DataFrame(respuesta)

    respuesta = df.to_dict(orient='records')

    return respuesta

In [ ]:
# Ejecutamos:
count_reviews('2011-01-01','2012-01-01')

***3° FUNCIÓN: DEF_GENRE***

In [ ]:
# Creamos un lista de listas
generos_lista = df_steam_games['genres'].tolist()


generos = []
for lista in generos_lista:
    lista = eval(lista)
    for gen in lista:
        generos.append(gen)

generos_unicos = list(set(generos))

print(generos_unicos)

In [ ]:
# Esta línea crea una lista vacía llamada playtimes para almacenar el tiempo de juego total de cada género.
playtimes = []
# Esta línea crea un bucle for que itera sobre cada género único.
for genero in generos_unicos:
    df_steam_games_generos = df_steam_games[df_steam_games['genres']
                                                    .apply(lambda generos: genero in generos)]
    lista_items = df_steam_games_generos['id'].tolist()

    df_user_items_final_items = df_user_items_final[df_user_items_final['item_id'].isin(lista_items)]
    df_user_items_final_items

    playtime = df_user_items_final_items['playtime_forever'].sum()

    playtimes.append(playtime)

In [ ]:
# Se diccionario que contiene los géneros y el tiempo de juego total de cada género para el usuario actual.
dicc_gnre = {'genero': generos_unicos, 'Time': playtimes}

In [ ]:
# Creamos un dataframe y ordenamos de forma ascendente para ver los juegos más jugados.
df_generos_times = pd.DataFrame(dicc_gnre)

df_generos_times = df_generos_times.sort_values('Time', ascending=False)

df_generos_times = df_generos_times.reset_index(drop=True)

In [ ]:
# Veamos la info
df_generos_times.head(5)

In [ ]:
# Esta línea agrega 1 al índice de cada fila del DataFrame df_generos_times.
df_generos_times.index = df_generos_times.index + 1

# Esta línea reinicia los índices del DataFrame df_generos_times usando el valor 0 como índice inicial
df_generos_times.reset_index(0, inplace=True)

df_generos_times.rename(columns={'index': 'Rank'}, inplace=True)
df_generos_times.to_csv('rankgenres.csv', index=False)

In [ ]:
def genre(genero):
    try:
        df = pd.read_csv('rankgenres.csv')
        rank = df[df['genero'] == genero]['Rank'].values[0]
    except:
        rank = 'Genero no existe'
    return rank


In [ ]:
genero = 'Action'
rank = genre(genero)
print(rank)

***4° FUNCIÓN: DEF USER_FORGENRE***

In [ ]:
# Top 5 de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.
def user_for_genre(genero):
  """
  Devuelve una lista de los 5 usuarios principales para el género especificado.

  Args:
    genero: El género.

  Returns:
    Una lista de los 5 usuarios principales para el género especificado.
  """

  # Obtiene los juegos del género especificado.
  df_steam_games_generos = df_steam_games[df_steam_games['genres'].apply(lambda generos: genero in generos)]
  juegos = df_steam_games_generos['id'].values

  # Obtiene los usuarios que han jugado los juegos del género especificado.
  df_user_items_final_genres = df_user_items_final[df_user_items_final['item_id'].isin(juegos)]

  # Agrupa los usuarios por ID y suma su tiempo de juego total para cada juego.
  df_top_users = df_user_items_final_genres[['user_id', 'user_url', 'playtime_forever']].groupby(['user_id', 'user_url']).sum('playtime_forever')

  # Obtiene los 5 usuarios principales ordenados por tiempo de juego total.
  df_top_users = df_top_users.sort_values('playtime_forever', ascending=False).head(5)

  # Convierte el DataFrame a una lista de diccionarios.
  respuesta = df_top_users.to_dict(orient='records')

  return respuesta


In [ ]:
user_for_genre('Action')

***5° FUNCIÓN: DEF_DEVELOPER***

In [ ]:
# Solicitado: cantidad` de items y `porcentaje` de contenido Free por año según empresa desarrolladora. Ej de salida.

# Supongamos que tienes un DataFrame llamado df con las columnas 'developer', 'price2', y 'año'

# Filtraremos los juegos con precio igual a 0.0 y agrupar por 'developer' y 'año'
free_games = df_steam_games[df_steam_games['price2'] == 0.0].groupby(['developer', 'Año']).size().reset_index(name='total gratis')

# Agrupamos todos los juegos por 'developer' y 'año' y contar el total

todos_los_juegos = df_steam_games.groupby(['developer', 'Año']).size().reset_index(name='total')

## merge ##
# Combinamos ambos DataFrames en uno solo
developer_año = pd.merge(todos_los_juegos, free_games, on=['developer', 'Año'], how='left').fillna(0)
developer_año.head(10)

In [ ]:
# Definimos una función para ver el contenido gratis:
def free_cont(fila):
    if(fila['total gratis']==0.0):
        return 0.0
    else:
        return round((fila['total gratis']/fila['total'])*100,2)

In [ ]:
developer_año['Contenido Free'] = developer_año.apply(free_cont, axis=1)

In [ ]:
# Imprimimos para saber el resultado:
developer_año.head(10)

In [ ]:
# Convierto archivo a csv
developer_año.to_csv('developerByAño.csv', index=False)

In [ ]:
# La función developer() devuelve un diccionario con la información del desarrollador especificado, o el mensaje No hay developer con ese nombre si el desarrollador no existe.
def developer(developer):
    df_developer = pd.read_csv('developerByAño.csv')
    df_developer = df_developer[df_developer['developer']==developer]

    if(len(df_developer)==0):
        respuesta = 'No hay developer con ese nombre'
    else:
        respuesta = df_developer.to_dict(orient='records')
    return respuesta

In [ ]:
developer('+7 Software')

**ENTORNO VIRTUAL**